# What are we doing?
Predict who survived (1) or died (0) based on input information.

Raw data is also availble on wikipedia.

In [618]:
import numpy as np
import pandas as pd
import collections
from random import shuffle, randint

from sklearn.cross_validation import train_test_split
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

Getting errors with sklearn? 

Scikit-learn requires:
<li>Python (>= 2.6 or >= 3.3),
<li>NumPy (>= 1.6.1),
<li>SciPy (>= 0.9).

Try:
conda install scikit-learn
OR
pip install -U scikit-learn

Still got probs?

python -m pip install --upgrade pip
pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose

In [605]:
#read in csv file
data = pd.read_csv("iris.csv", header=None)
trainPercent = 30 #must be between 0 and 100
validationPercent = 35
numberRowsOfEachClass = 50
data.head() #look at first 5 rows of the data

,0,1,2,3,4
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


# Useful functions
<li>date_frame.head()
<li>data.columns
<li>data_frame.shape


In [606]:
#Grab desired inputs to test one. Build a dataframe from them.
dataInputs= data.loc[:,0:3]
dataInputs.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [607]:
#Grab the expected outputs. (This is supervised learning)
y1=[]
y2=[]
y3=[]
for i in range(0,numberRowsOfEachClass):
    y1.append([1,0,0]);#setosa
    y2.append([0,1,0]);#versicolor
    y3.append([0,0,1]);#virginica
expectedOutput=np.concatenate([y1,y2,y3])


# Breakdown the input data into test and train

In [608]:
#Hint: use train_test_split
inputTrain, inputTest, expectedOutputTrain, expectedOutputTest = train_test_split(dataInputs, expectedOutput, train_size = trainPercent/100, random_state=42)
validateTest, inputTrain, validateOutputTest, expectedOutputTrain = train_test_split(inputTrain, expectedOutputTrain, train_size = validationPercent/100, random_state=42)
#inputTest.head()
print(validateTest.shape)
print(validateOutputTest.shape)
print(inputTrain.shape)
print(expectedOutputTrain.shape)
print(inputTest.shape)
print(expectedOutputTest.shape)

def getClassCountTotals(output):
    d = collections.OrderedDict()
    for a in output:
        t = tuple(a)
        if t in d:
            d[t] += 1
        else:
            d[t] = 1

    result = []
    for (key, value) in d.items():
        result.append(list(key) + [value])

    B = np.asarray(result)
    print(B)
    for Bi in B:
        if Bi[0] == 1:
            setosaTotal = Bi[3]
        elif Bi[1] == 1:
            versicolorTotal = Bi[3]
        elif Bi[2] == 1:
            virginicaTotal = Bi[3]
    return setosaTotal, versicolorTotal, virginicaTotal
print(expectedOutputTest)
print("ValidateTest",validateOutputTest)
setosaTotal, versicolorTotal, virginicaTotal = getClassCountTotals(expectedOutputTest)
validationCounts = getClassCountTotals(validateOutputTest)
print(setosaTotal)
print(versicolorTotal)
print(virginicaTotal)

(15, 4)
(15, 3)
(30, 4)
(30, 3)
(105, 4)
(105, 3)
[[0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]]
Vali

# Get weight matrix

In [609]:
#Build our least squares classifier for 2 classes
D = inputTrain.shape[1] + 1 #num of attributes, +1 is for the intercept (column of 1s)
K = expectedOutput.shape[1]

In [610]:
expectedOutputTrain = np.asarray(expectedOutputTrain)
expectedOutputTest = np.asarray(expectedOutputTest)
inputTrain = np.asarray(inputTrain)
inputTest = np.asarray(inputTest)
validateTest = np.asarray(validateTest)
validateOutputTest = np.asarray(validateOutputTest)

In [611]:
sum1 = 0
sum2 = 0
def createWeightMatrix(x, y, lmbda):
    numRows = x.shape[0]
    new_col = np.ones((numRows,1))
    augmentedX = np.c_[new_col, x]
    sum1 = np.dot(augmentedX.T, augmentedX) + lmbda
    sum2 = np.dot(augmentedX.T, y)
    W = np.dot(np.linalg.inv(sum1), sum2)
    return W

In [612]:
def classificationTest(xTest, yTest, W, isValidating):  
    returnVals = []
    total = yTest.shape[0]
    predicted = np.zeros((total, K))
    i=0
    setosaCorrect=versicolorCorrect=virginicaCorrect=totalCorrect=0
    correctPredictionCol = [0]*total
    for i in range(total):      
        x = xTest[i]        
        x = np.append(1,x)         
        x = x.reshape(1,D)        
        values = np.dot(W.T,x.T)
        values = values.T
        values = values.flatten()
        maxIndex = np.argmax(values)
        if yTest[i][maxIndex] == 1:
            if maxIndex == 0:
                setosaCorrect += 1
            elif maxIndex == 1:
                versicolorCorrect += 1
            elif maxIndex == 2:
                virginicaCorrect += 1
            correctPredictionCol[i] = 1
        predicted[i][maxIndex] = 1
    totalCorrect = setosaCorrect + versicolorCorrect + virginicaCorrect
    print(totalCorrect)
    totalAccuracy=totalCorrect/float(total)*100
    if isValidating == False:
        setosaAccuracy = setosaCorrect/float(setosaTotal)*100
        versicolorAccuracy = versicolorCorrect/float(versicolorTotal)*100
        virginicaAccuracy = virginicaCorrect/float(virginicaTotal)*100
        print("setosa misclassification error = ", 100 - setosaAccuracy)
        print("versicolor misclassification error = ", 100 - versicolorAccuracy)
        print("virginica misclassification error = ", 100 - virginicaAccuracy)
    else:
        setosaAccuracy = setosaCorrect/float(validationCounts[0])*100
        versicolorAccuracy = versicolorCorrect/float(validationCounts[1])*100
        virginicaAccuracy = virginicaCorrect/float(validationCounts[2])*100
        print("setosa misclassification error = ", 100 - setosaAccuracy)
        print("versicolor misclassification error = ", 100 - versicolorAccuracy)
        print("virginica misclassification error = ", 100 - virginicaAccuracy)
    print("total misclassification error = ", 100 - totalAccuracy)
    print(total)
    returnVals.append(100-totalAccuracy)
    returnVals.append(predicted)
    returnVals.append(correctPredictionCol)
    return returnVals

In [613]:
#cross validation
lambdaValues = [100000, 100, 0,.1 , .3, .01, .03, .001, .003, .0001, .0003]
lambdaIndexOfMinError = -1
minError = 101
i = 0
for l in lambdaValues :
    print("lambda val?",l)
    W=0
    W = createWeightMatrix(inputTrain, expectedOutputTrain, l)
    print("W",W)
    error, predictedClasses, correctPredCol = classificationTest(validateTest, validateOutputTest, W, True)
    print("error ",error)    
    if(error < minError):
        minError = error
        lambdaIndexOfMinError = i
    i+=1
print("smalled index of error for lambda...", lambdaIndexOfMinError)

Wtest = createWeightMatrix(inputTrain, expectedOutputTrain, lambdaValues[lambdaIndexOfMinError])

lambda val? 100000
W [[-0.06950198  0.77346239 -0.82847999]
 [ 0.11692571  0.14632405 -0.11241457]
 [ 0.24265431 -0.47654072  0.33822911]
 [-0.27034263  0.26729181  0.03202513]
 [-0.01973429 -0.71053646  0.57064163]]
12
setosa misclassification error =  0.0
versicolor misclassification error =  33.33333333333334
virginica misclassification error =  12.5
total misclassification error =  20.0
15
error  20.0
lambda val? 100
W [[-0.0682414   0.77465933 -0.82700266]
 [ 0.11675663  0.14616351 -0.11261272]
 [ 0.24253734 -0.47665179  0.33809203]
 [-0.27037511  0.26726097  0.03198707]
 [-0.01955534 -0.71036655  0.57085135]]
12
setosa misclassification error =  0.0
versicolor misclassification error =  33.33333333333334
virginica misclassification error =  12.5
total misclassification error =  20.0
15
error  20.0
lambda val? 0
W [[ 0.2907524   1.11552954 -0.40628193]
 [ 0.06860371  0.10044156 -0.16904526]
 [ 0.20922678 -0.50828068  0.29905391]
 [-0.27962492  0.25847813  0.0211468 ]
 [ 0.03140496

In [614]:
#testing
error, predictedClasses, correctPredictionCol = classificationTest(inputTest, expectedOutputTest, Wtest, False)
print(error)

89
setosa misclassification error =  0.0
versicolor misclassification error =  30.303030303030297
virginica misclassification error =  18.75
total misclassification error =  15.23809523809524
105
15.23809523809524


In [615]:
actual = pd.DataFrame(expectedOutputTest)
predictedDF = pd.DataFrame(predictedClasses)
correct = pd.DataFrame(correctPredictionCol)
frames = [predictedDF, actual, correct]
modelTest = pd.concat(frames, axis=1)
modelTest.columns = ["Predicted","Predicted", "Predicted", "Actual", "Actual", "Actual", "Correct"]
modelTest

,Predicted,Predicted,Predicted,Actual,Actual,Actual,Correct
0,0,1,0,0,1,0,1
1,1,0,0,1,0,0,1
2,0,1,0,0,0,1,0
3,0,0,1,0,1,0,0
4,0,1,0,0,1,0,1
5,1,0,0,1,0,0,1
6,0,0,1,0,1,0,0
7,0,0,1,0,0,1,1
8,0,1,0,0,1,0,1
9,0,1,0,0,1,0,1


In [616]:
actualAsClassNumber = [0]*expectedOutputTest.shape[0]
predictedAsClassNumber = [0]*expectedOutputTest.shape[0]
for i in range(expectedOutputTest.shape[0]):
    actualAsClassNumber[i] = expectedOutputTest[i].tolist().index(1)
    predictedAsClassNumber[i] = predictedClasses[i].tolist().index(1.0)
confusion_matrix(actualAsClassNumber, predictedAsClassNumber)

array([[40,  0,  0],
       [ 0, 23, 10],
       [ 0,  6, 26]])

In [617]:
clf = svm.SVC(kernel='linear', C=1)
expectedOutputSVM = data.loc[:,4]
inputTrain, inputTest, expectedOutputTrain, expectedOutputTest = train_test_split(dataInputs, expectedOutputSVM, train_size = trainPercent/100, random_state=42)
clf.fit(inputTrain, expectedOutputTrain)
accuracy = clf.score(inputTest, expectedOutputTest)
print(accuracy)

0.980952380952


In [622]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier

In [626]:
#SVM One vs all classifier
clf = Pipeline([
    ('clf', OneVsRestClassifier(LinearSVC()))])
clf.fit(inputTrain, expectedOutputTrain)
accuracy = clf.score(inputTest, expectedOutputTest)
print(accuracy)

0.961904761905
